In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col
from pyspark.sql.types import StructField,StructType,StringType

In [0]:
 %sh
 rm -r /dbfs/delta_lab
 mkdir /dbfs/delta_lab
 wget -O /dbfs/delta_lab/products.csv https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv

--2024-07-03 13:04:23--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14372 (14K) [text/plain]
Saving to: ‘/dbfs/delta_lab/products.csv’

     0K .......... ....                                       100% 2.69M=0.005s

2024-07-03 13:04:24 (2.69 MB/s) - ‘/dbfs/delta_lab/products.csv’ saved [14372/14372]



In [0]:
df = spark.read.load('dbfs:/delta_lab/products.csv', format='csv', header=True)
display(df.limit(num=10))

ProductID,ProductName,Category,ListPrice
771,"Mountain-100 Silver, 38",Mountain Bikes,3399.9900
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.9900


In [0]:
%sh
rm -rf $deltaTablePath

In [0]:
deltaTablePath="/delta1/products-delta"

df.write.format(source="delta").save(path=deltaTablePath)

In [0]:
dbutils.fs.ls("dbfs:/delta")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
%sh
ls /dbfs/delta/products-delta

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
deltaTable=DeltaTable.forPath(sparkSession=spark,path=deltaTablePath)

In [0]:
deltaTable.update(
    condition="ProductID == 771",
    set={"ListPrice":"ListPrice * 0.9" }
)

In [0]:
deltaTable.toDF().show(n=10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      781|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



### See the Original Version of Delta Table

In [0]:
newDF=spark.read.format("delta").option("versionAsOf",0).load(path=deltaTablePath)

In [0]:
newDF.show(n=10)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [0]:
deltaTable.history(limit=10).show(n=10,truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 2                                                                                                                                                                                                                                                                                                                               
 timestamp           | 2024-07-03 13:06:03                                                                                                                                                                                                                                                                              

### Create an External Table

In [0]:
spark.sql(
  sqlQuery="""
  CREATE DATABASE 
    Adventureworks
  """
)

DataFrame[]

In [0]:
spark.sql(
    sqlQuery="""
    CREATE TABLE 
        AdventureWorks.ProductsExternal
    USING
        delta
    LOCATION
        '{0}'
        """.format(deltaTablePath))

DataFrame[]

In [0]:
spark.sql(sqlQuery="""
          DESCRIBE EXTENDED 
            AdventureWorks.ProductsExternal
          """).show(truncate=False)

+----------------------------+-------------------------------------------+-------+
|col_name                    |data_type                                  |comment|
+----------------------------+-------------------------------------------+-------+
|ProductID                   |string                                     |NULL   |
|ProductName                 |string                                     |NULL   |
|Category                    |string                                     |NULL   |
|ListPrice                   |string                                     |NULL   |
|                            |                                           |       |
|# Delta Statistics Columns  |                                           |       |
|Column Names                |ProductID, ProductName, Category, ListPrice|       |
|Column Selection Method     |first-32                                   |       |
|                            |                                           |       |
|# D

In [0]:
dbutils.fs.ls(path="dbfs:/delta/products-delta")

[FileInfo(path='dbfs:/delta/products-delta/_delta_log/', name='_delta_log/', size=0, modificationTime=1720008020000),
 FileInfo(path='dbfs:/delta/products-delta/deletion_vector_9b12f505-1a2f-4cfc-847f-e0e317da1b00.bin', name='deletion_vector_9b12f505-1a2f-4cfc-847f-e0e317da1b00.bin', size=43, modificationTime=1720008900000),
 FileInfo(path='dbfs:/delta/products-delta/deletion_vector_b6659ece-8ddd-4697-b7a8-13f66ac6b6e6.bin', name='deletion_vector_b6659ece-8ddd-4697-b7a8-13f66ac6b6e6.bin', size=43, modificationTime=1720011960000),
 FileInfo(path='dbfs:/delta/products-delta/part-00000-2050bf2c-d4a6-440d-9fb9-65e66117bcdc.c000.snappy.parquet', name='part-00000-2050bf2c-d4a6-440d-9fb9-65e66117bcdc.c000.snappy.parquet', size=1153, modificationTime=1720008901000),
 FileInfo(path='dbfs:/delta/products-delta/part-00000-23faeef8-d6fe-4a8c-a3d7-182db2a3f3de.c000.snappy.parquet', name='part-00000-23faeef8-d6fe-4a8c-a3d7-182db2a3f3de.c000.snappy.parquet', size=1158, modificationTime=1720011962000)

In [0]:
%sql
USE 
  Adventureworks;
SELECT
  *
FROM 
  ProductsExternal;

ProductID,ProductName,Category,ListPrice
772,"Mountain-100 Silver, 42",Mountain Bikes,3399.9900
773,"Mountain-100 Silver, 44",Mountain Bikes,3399.9900
774,"Mountain-100 Silver, 48",Mountain Bikes,3399.9900
775,"Mountain-100 Black, 38",Mountain Bikes,3374.9900
776,"Mountain-100 Black, 42",Mountain Bikes,3374.9900
777,"Mountain-100 Black, 44",Mountain Bikes,3374.9900
778,"Mountain-100 Black, 48",Mountain Bikes,3374.9900
779,"Mountain-200 Silver, 38",Mountain Bikes,2319.9900
780,"Mountain-200 Silver, 42",Mountain Bikes,2319.9900
781,"Mountain-200 Silver, 46",Mountain Bikes,2319.9900


### Create a Managed Table

In [0]:
df.write.format(source="delta").saveAsTable(name="AdventureWorks.ProductsManaged")

In [0]:
spark.sql(sqlQuery="""
          DESCRIBE EXTENDED 
            AdventureWorks.ProductsManaged;
          """).show(truncate=False)

+----------------------------+-----------------------------------------------------------+-------+
|col_name                    |data_type                                                  |comment|
+----------------------------+-----------------------------------------------------------+-------+
|ProductID                   |string                                                     |NULL   |
|ProductName                 |string                                                     |NULL   |
|Category                    |string                                                     |NULL   |
|ListPrice                   |string                                                     |NULL   |
|                            |                                                           |       |
|# Delta Statistics Columns  |                                                           |       |
|Column Names                |ProductID, ProductName, Category, ListPrice                |       |
|Column Se

### Comparing two tables

In [0]:
%sql 
USE Adventureworks;
SHOW TABLES;

database,tableName,isTemporary
adventureworks,productsexternal,false
adventureworks,productsmanaged,false


In [0]:
%sh 
echo "External Table:"
ls /dbfs/delta/products-delta
echo
echo "Managed Table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

External Table:
_delta_log
deletion_vector_9b12f505-1a2f-4cfc-847f-e0e317da1b00.bin
deletion_vector_b6659ece-8ddd-4697-b7a8-13f66ac6b6e6.bin
part-00000-2050bf2c-d4a6-440d-9fb9-65e66117bcdc.c000.snappy.parquet
part-00000-23faeef8-d6fe-4a8c-a3d7-182db2a3f3de.c000.snappy.parquet
part-00000-a7ece34d-3d32-4368-8fbe-5a8bcaab6c84.c000.snappy.parquet

Managed Table:
_delta_log
part-00000-02905c28-6ef6-4d7f-b8b3-6f009ee329a2.c000.snappy.parquet


### Delete Both the Tables

In [0]:
%sql

USE Adventureworks;
DROP TABLE IF EXISTS
  productsexternal;
DROP TABLE IF EXISTS
  productsmanaged;

SHOW TABLES;

database,tableName,isTemporary


### The files for External Tables remains while the file for the managed table are deleted automatically

In [0]:
%sh 
echo "External Table:"
ls /dbfs/delta/products-delta
echo
echo "Managed Table:"
ls /dbfs/user/hive/warehouse/adventureworks.db/productsmanaged

External Table:
_delta_log
deletion_vector_9b12f505-1a2f-4cfc-847f-e0e317da1b00.bin
deletion_vector_b6659ece-8ddd-4697-b7a8-13f66ac6b6e6.bin
part-00000-2050bf2c-d4a6-440d-9fb9-65e66117bcdc.c000.snappy.parquet
part-00000-23faeef8-d6fe-4a8c-a3d7-182db2a3f3de.c000.snappy.parquet
part-00000-a7ece34d-3d32-4368-8fbe-5a8bcaab6c84.c000.snappy.parquet

Managed Table:


### Use Delta Tables for Streaming Data

In [0]:
 %sh
 rm -r /dbfs/device_stream
 mkdir /dbfs/device_stream
 wget -O /dbfs/device_stream/devices1.json https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json

rm: cannot remove '/dbfs/device_stream': No such file or directory
--2024-07-03 14:08:39--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 296 [text/plain]
Saving to: ‘/dbfs/device_stream/devices1.json’

     0K                                                       100% 3.50M=0s

2024-07-03 14:08:39 (3.50 MB/s) - ‘/dbfs/device_stream/devices1.json’ saved [296/296]



In [0]:
inputPath="dbfs:/device_stream/"
jsonSchema=StructType(
    [
        StructField(name="device",dataType=StringType(),nullable=False),
        StructField(name="status",dataType=StringType(),nullable=False)
    ]
)

In [0]:
iotstream=spark.readStream.schema(schema=jsonSchema).option(
    key="maxFilesPerTrigger",value=1).json(path=inputPath)

print("Source Stream Created")

Source Stream Created


In [0]:
deltaStreamTablePath="/delta/iotdevicedata"

checkPointPath="dbfs/delta/checkpoint"

deltaStream=iotstream.writeStream.format("delta").\
    option(key="checkpointLocation",value=checkPointPath).\
    start(path=deltaStreamTablePath)

In [0]:
df=spark.read.format(source="delta").load(deltaStreamTablePath)
display(df)

device,status
Dev1,ok
Dev1,ok
Dev1,ok
Dev2,error
Dev1,ok
Dev1,error
Dev2,ok
Dev2,error
Dev1,ok


In [0]:
# create a catalog table based on the streaming sink
spark.sql(
    sqlQuery="""
            CREATE TABLE 
                IotDeviceData 
            USING 
                delta
            LOCATION
                '{0}';
          """.format(deltaStreamTablePath))

DataFrame[]

In [0]:
%sql
SELECT 
  *
FROM 
  IotDeviceData;

device,status
Dev1,ok
Dev1,ok
Dev1,ok
Dev2,error
Dev1,ok
Dev1,error
Dev2,ok
Dev2,error
Dev1,ok


#### Adding some Fresh Device to The Stream

In [0]:
 %sh
 wget -O /dbfs/device_stream/devices2.json https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices2.json

--2024-07-03 14:29:25--  https://raw.githubusercontent.com/MicrosoftLearning/mslearn-databricks/main/data/devices2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229 [text/plain]
Saving to: ‘/dbfs/device_stream/devices2.json’

     0K                                                       100% 2.81M=0s

2024-07-03 14:29:25 (2.81 MB/s) - ‘/dbfs/device_stream/devices2.json’ saved [229/229]



In [0]:
%sql
SELECT 
  *
FROM
  IotDeviceData;

device,status
Dev1,ok
Dev1,ok
Dev1,ok
Dev2,error
Dev1,ok
Dev1,error
Dev2,ok
Dev2,error
Dev1,ok
Dev1,ok


In [0]:
## Stopping the Stream

deltaStream.stop()